In [ ]:
import torch 

torch.manual_seed(0)

print(torch.get_rng_state().shape)

In [ ]:
import torch
from tqdm import tqdm, trange
from matplotlib.gridspec import GridSpec
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from devinterp.slt.forms import get_osculating_circle
from skimage.measure import EllipseModel

# Define a function to create a circle in 2D
def create_circle(radius, center=(0, 0), num_points=100):
    theta = np.linspace(0, 2 * np.pi, num_points)
    x = radius * np.cos(theta) + center[0]
    y = radius * np.sin(theta) + center[1]

    return x, y

# Define a function to rotate the circle around the z-axis
def rotate_circle(x, y, z, angle):
    # Define the rotation matrix
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle), 0],
                                [np.sin(angle), np.cos(angle), 0],
                                [0, 0, 1]])
    
    # Embed the 2D circle in 3D by adding a z=0 coordinate
    points_2d = np.array([x, y, z])
    
    # Apply the rotation
    points_3d = rotation_matrix @ points_2d
    return points_3d

def fit_ellipse(curve):
    # Create the model
    model = EllipseModel()
    model.estimate(curve)
    print(model.params)

    return model


# Create the 2D circle
radius = 1
num_steps = 100
x, z = create_circle(radius, num_points=num_steps)

num_turns = 5
# fig, axes = plt.subplots(2, num_turns, figsize=(4 * num_turns, 5))

fig = plt.figure(figsize=(20, 10))
gs = GridSpec(2, num_turns)
axes = [fig.add_subplot(gs[0, i]) for i in range(num_turns)]

extra = 10

angles = np.linspace(0, np.pi / 2, num_turns * extra)

max_z_evolute_centers = []

for i, angle in enumerate(angles):
    # Set the angle of rotation around the z-axis
    x_rotated, y_rotated, z_rotated = rotate_circle(x, np.zeros_like(x), z, angle)

    xz = np.stack([x_rotated, z_rotated], axis=1)
    circles = [get_osculating_circle(xz, t) for t in range(num_steps)]
    centers = np.array([center for center, _ in circles])
    max_z_evolute_centers.append(max([center[1] for center, _ in circles]))

    if (i % extra) == 0:
        ax = axes[i // extra]
        # Plot the osculating circles
        for center, radius in circles:
            _x, _z = create_circle(radius, center)
            ax.plot(_x, _z, label='Original circle', color="gray", alpha=0.1)

        # Plot the centers
        ax.scatter(centers[:, 0], centers[:, 1], color='k', s=10)
        ax.plot(x_rotated, z_rotated, label='Rotated circle', color='r')
        ax.set_title(f'Angle: {angle:.2f}')

        ellipse = fit_ellipse(xz)
        xc, yc, a, b, theta = ellipse.params

        c = np.sqrt(a**2 - b**2)
        major_vec = np.array([a * np.cos(theta), a * np.sin(theta)]) 
        focus1 = np.array([xc, yc]) + c * major_vec / np.linalg.norm(major_vec)
        focus2 = np.array([xc, yc]) - c * major_vec / np.linalg.norm(major_vec)

        # Plot the ellipse
        ax.scatter(focus1[0].item(), focus1[1].item(), color='g', s=20, marker='x')
        ax.scatter(focus2[0].item(), focus2[1].item(), color='g', s=20, marker='x')

        # t = np.linspace(0, 2 * np.pi, 100)
        # curve = ellipse.predict_xy(t)
        # ax.plot(curve[:, 0], curve[:, 1], label='Fitted ellipse', color='g')  

for ax in axes:
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_aspect('equal')

bottom_ax = fig.add_subplot(gs[1, :])
bottom_ax.plot(angles, max_z_evolute_centers) 
bottom_ax.set_xlabel('Angle')
bottom_ax.set_ylabel('Max z-coordinate of the evolute center')
plt.tight_layout()
fig.set_facecolor('w')

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import pairwise_distances
from torch import nn

def create_timeseries(s, a, delta, t_range):
    """
    Create a timeseries of 3D vectors based on the given parameters.

    Parameters:
    - s: Tuple or list of s parameters (s_1, s_2, s_3)
    - a: Tuple or list of a parameters (a_1, a_2, a_3)
    - t_range: Array of time values over which to compute the coordinates

    Returns:
    - A numpy array of shape (len(t_range), 3) containing the 3D vectors for each time t
    """
    # Preallocate the array
    timeseries = np.zeros((len(t_range), 3))

    # Compute the coordinates for each time t
    for i, t in enumerate(t_range):
        for j in range(3):
            timeseries[i, j] = s[j] * np.exp(s[j] * (t - delta[j])) / (np.exp(s[j] * (t - delta[j])) - 1 + s[j]/a[j])

    return timeseries

s = (3.0, 2.0, 1.0)  # s_1, s_2, s_3
a = (0.1, 0.1, 0.1)  # a_1, a_2, a_3
delta = (0.0, 2.0, 4.0)
num_steps = 1000
t_range = np.linspace(0, 12, num_steps)  # Time range from 0 to 1 with 100 steps

timeseries = create_timeseries(s, a, delta, t_range)

pca = PCA(n_components=3)
samples = pca.fit_transform(timeseries)

marked_cusp_data = [ { "step" : 197, "influence_start" : 190, "influence_end" : 210 }, 
                    { "step" : 425, "influence_start" : 390, "influence_end" : 450 }]
                    #{ "step" : 498, "influence_start" : 450, "influence_end" : 550 }]

# Plotting
plt.figure(figsize=(10, 7), facecolor='white')
plt.plot(t_range, timeseries[:,0], label=r'$\alpha = 1$')
plt.plot(t_range, timeseries[:,1], label=r'$\alpha = 2$')
plt.plot(t_range, timeseries[:,2], label=r'$\alpha = 3$')
plt.axvline(marked_cusp_data[0]["step"] / num_steps * 12, color='black', linestyle=':', lw=2)
plt.axvline(marked_cusp_data[1]["step"] / num_steps * 12, color='black', linestyle=':', lw=2)
#plt.axvline(marked_cusp_data[2]["step"] / num_steps * 12, color='black', linestyle=':', lw=1, alpha=0.2)

plt.xlabel('Time (t)')
plt.ylabel('Values')
plt.title('Logistic curves')
plt.legend()
plt.grid(True)
plt.show()


class KVars(KMeans):
    def __init__(self, n_clusters=8, max_iter=300, tol=1e-4, verbose=0, random_state=None, copy_x=True):
        super().__init__(
            n_clusters=n_clusters,
            init='random',
            max_iter=max_iter,
            tol=tol,
            verbose=verbose,
            random_state=random_state,
            copy_x=copy_x,
            n_init=1
        )
    
    def fit(self, X, sample_weight=None):
        # Initialize centroids by partitioning the dataset into equal-sized batches
        # and computing the mean & std of each batch
        n_samples, n_features = X.shape
        n_clusters = self.n_clusters
        init_n_samples_per_centroid = n_samples // n_clusters

        init_centers = np.zeros((n_clusters, n_features))
        init_radii = np.zeros(n_clusters)

        for c in range(n_clusters):
            batch = X[c * init_n_samples_per_centroid: (c + 1) * init_n_samples_per_centroid, :]
            init_centers[c, :] = np.mean(batch, axis=0)
            init_radii[c] = np.std(batch)
            print(np.mean(batch, axis=0))
        
        if self.verbose:
            print(f"Initializing centroids using {init_n_samples_per_centroid} samples per cluster at {init_centers} with {init_radii} radii.")

        self.cluster_centers_ = init_centers
        self.cluster_radii_ = init_radii
        # boundaries = list(np.linspace(0, n_samples, init_n_samples_per_centroid).astype(int))[1:] + [n_samples]

        def get_boundaries():
            distances = np.zeros((n_clusters, n_samples))
            for c in range(n_clusters):
                distances[c] = np.linalg.norm(X - self.cluster_centers_[c], axis=1)

            distances_from_radii = np.abs(distances - self.cluster_radii_.reshape(-1, 1))

            boundaries = [0]
            cluster = 0
            for t in range(n_samples):
                if distances_from_radii[cluster][t] > distances_from_radii[cluster + 1][t]:
                    boundaries.append(t)
                    cluster += 1

                if cluster + 1 == n_clusters:
                    boundaries.append(n_samples)
                    return np.array(boundaries)
                
            return np.array(boundaries)
        
        centers = nn.Parameter(torch.tensor(self.cluster_centers_))
        radii = nn.Parameter(torch.tensor(self.cluster_radii_))

        Xtens = torch.tensor(X)
        optimizer = torch.optim.Adam([centers, radii], lr=0.001)

        for i in range(self.max_iter):
            # Assign clusters based on closest centroid
            boundaries = get_boundaries()
            
            labels = []

            i = 0
            for b, boundary in enumerate(boundaries):
                while i < boundary:
                    labels.append(b)
                    i += 1

            labels = np.array(labels)

            for k in range(n_clusters):
                cluster_points = Xtens[labels == k]
                
                if len(cluster_points) > 0:
                    # Calculate the variance within the cluster
                    within_cluster_distances = torch.norm(cluster_points - centers[k], dim=1) 
                    loss = ((within_cluster_distances - radii[k]) ** 2).sum() + 0 * within_cluster_distances.sum()
                    optimizer.zero_grad()  
                    loss.backward()
                    optimizer.step()

            self.cluster_centers_ = centers.detach().numpy()
            self.cluster_radii_ = radii.detach().numpy()

            if self.verbose:
                print(f"Iteration {i}: {loss.item()}")
                print(f"Centers: {self.cluster_centers_}")
                print(f"Radii: {self.cluster_radii_}")

        self.boundaries_ = boundaries
        # self.inertia_ = np.sum([cluster_variances[k] * len(X[labels == k]) for k in range(n_clusters)])
        self.n_iter_ = i

        return self 

fig, axes = plt.subplots(1, 3, figsize=(15, 5), facecolor='white')
# _kvars = KVars(n_clusters=3, random_state=0, verbose=True, max_iter=3000).fit(samples)

for i, (ax, (pc1, pc2)) in enumerate(zip(axes, [(0, 1), (1, 2), (0, 2)])):
    ax.plot(samples[:, pc1], samples[:, pc2])
    ax.set_xlabel(f'PC {pc1 + 1}')
    ax.set_ylabel(f'PC {pc2 + 1}')


    for t in range(1, num_steps-1):
        # dpc1_dpc2 = (samples[t+1, [pc1, pc2]] - samples[t-1, [pc1, pc2]]) * 100
        
        # ax.quiver(samples[t, pc1], samples[t, pc2], dpc1_dpc2[0], dpc1_dpc2[1], angles='xy', scale_units='xy', scale=100, color='gray', alpha=0.1)
        center, radius = get_osculating_circle(samples[:, [pc1, pc2]], t)
        circ = plt.Circle(center, radius, fill=False, color='gray', alpha=0.1)
        ax.add_artist(circ)


    for c, r in zip(_kvars.cluster_centers_, _kvars.cluster_radii_):
        ax.scatter(c[pc1], c[pc2], color='g', s=10)
        circ = plt.Circle(c, r, fill=False, color='g', alpha=0.5)
        ax.add_artist(circ)
        ax.set_xlim(-3, 3)
        ax.set_ylim(-3, 3)

print(_kvars.cluster_centers_)
plt.tight_layout()
plt.show()

In [ ]:
import itertools
from math import isnan
from sklearn.decomposition import PCA

from icl.analysis.smoothing import gaussian_filter1d_variable_sigma

torch.manual_seed(1)

class ToyModelOfForms(nn.Module):
    def __init__(self, num_forms=3, num_features=3, distance_scale=10, onset_times=1000, char_timescales=2000, thickness=2, radii=2):
        super().__init__()
        self.num_forms = num_forms
        self.num_features = num_features
        self.forms = nn.Parameter(torch.randn(num_forms, num_features) * distance_scale)
        
        self.onset_times = nn.Parameter(torch.rand(num_forms) * onset_times)
        self.char_timescales = nn.Parameter(torch.rand(num_forms) * char_timescales)
        
        self.thickness = nn.Parameter(torch.rand(num_forms) * thickness)
        self.radii = nn.Parameter(torch.rand(num_forms) * radii)
        

    def forward(self, x):
        raise NotImplementedError
    
TYPE = "Sigmoidal in distance" # "Gaussian in time"
DISTANCE_SCALE = 10
ONSET_TIMES = 1000
CHAR_TIMESCALES = 3000
THICKNESS = 2
RADII = 2
NUM_FORMS, NUM_OG_FEATURES = 7, 100
toy_model = ToyModelOfForms(NUM_FORMS, NUM_OG_FEATURES, DISTANCE_SCALE, ONSET_TIMES, CHAR_TIMESCALES, THICKNESS, RADII)
pos = torch.randn(NUM_OG_FEATURES)
# pos = nn.Parameter(torch.randn(NUM_OG_FEATURES))
NUM_STEPS = 1000
LR = 0.001 # 0.001
NOISE = 0 # 0.05
MIN_SMOOTHING, MAX_SMOOTHING = 0, 0

NUM_FEATURES = 5
init_details = "random"
details =  f"\n({TYPE}, {NUM_FORMS} forms, {NUM_OG_FEATURES} features, {NUM_FEATURES} PCs, {NUM_STEPS} steps, LR={LR}, noise={NOISE}, smoothing={MIN_SMOOTHING}..{MAX_SMOOTHING}, init={init_details})"

positions = np.zeros((NUM_STEPS, NUM_OG_FEATURES))
strengths = np.zeros((NUM_STEPS, NUM_FORMS))
potentials = np.zeros((NUM_STEPS, NUM_FORMS))

def gaussian_strength(t, onset_time, timescale):
    return torch.exp(-(t - onset_time).pow(2) / (2 * timescale.pow(2)))

def lennart_jones_potential(form_pos, curr_pos, thickness, radius):
    # Doesn't work at all.
    return (4 * thickness * ((radius / (form_pos - curr_pos)).pow(12) - 2 * (radius / (form_pos - curr_pos)).pow(6))).sum()

def sigmoidal_in_distance(form_pos, curr_pos, thickness, radius):
    return (1-(1 / (1 + torch.exp(-((form_pos - curr_pos).pow(2) - radius) / thickness)))).sum()

# pos.requires_grad = True

for t in range(NUM_STEPS):
    positions[t] = pos.detach().numpy()
    
    delta = torch.zeros_like(pos)

    for alpha in range(toy_model.num_forms):

        _delta = (toy_model.forms[alpha] - pos)

        # Gaussian update
        if TYPE == "Gaussian in time":
            form_strength = gaussian_strength(t, toy_model.char_timescales[alpha], toy_model.onset_times[alpha])

            # Sigmoidal update
        elif TYPE == "Sigmoidal in distance":
            form_strength = sigmoidal_in_distance(toy_model.forms[alpha], pos, toy_model.thickness[alpha], toy_model.radii[alpha])
            
        delta += form_strength * _delta

        # Lennart-Jones
        # pos.grad = torch.zeros_like(pos)
        # potential = lennart_jones_potential(toy_model.forms[alpha], pos, toy_model.thickness[alpha], toy_model.radii[alpha])
        # potential.backward()
        # # print(potential, pos.grad)
        # # Clip the gradient
        # pos.grad = torch.clamp(pos.grad, -1, 1)

        # delta += pos.grad

        strengths[t, alpha] = form_strength
        potentials[t, alpha] = _delta.pow(2).sum().sqrt().item()


    pos = pos + LR * delta + NOISE * torch.randn_like(pos) 

pca = PCA(n_components=NUM_FEATURES)
positions = pca.fit_transform(positions)


if MIN_SMOOTHING == MAX_SMOOTHING == 0:
    smoothed_positions = positions
else:
    smoothed_positions = gaussian_filter1d_variable_sigma(positions, np.linspace(MIN_SMOOTHING, MAX_SMOOTHING, NUM_STEPS), axis=0)

# smoothed_positions = pca.transform(smoothed_positions)

# PC components
fig, axes = plt.subplots(1, NUM_FEATURES, figsize=(5 * NUM_FEATURES, 5), facecolor='white')

for i, ax in enumerate(axes):
    ax.scatter(range(NUM_STEPS), positions[:, i], label=f'PC {i + 1}', color='k', alpha=0.1)
    ax.plot(smoothed_positions[:, i], label=f'PC {i + 1} (smoothed)', color='r', alpha=0.5, lw=2)
    ax.set_xlabel('Time')
    ax.set_ylabel(f'PC {i + 1}')

fig.suptitle(f'Principal components over time {details}', fontsize=16)

plt.tight_layout()

# Potentials

fig, axes = plt.subplots(1, NUM_FORMS, figsize=(5 * NUM_FORMS, 5), facecolor='white')

for alpha, ax in enumerate(axes):
    ax.plot(potentials[:, alpha], label=f'Potential {alpha + 1}')   
    axtwin = ax.twinx()
    axtwin.plot(strengths[:, alpha], label=f'Strength {alpha + 1}', linestyle=':', color='r')
    # axtwin.set_ylabel(f'Strength {alpha + 1}')
    ax.set_xlabel('Time')
    # ax.set_ylabel(f'Potential {alpha + 1}')
    ax.set_title(f'Form {alpha + 1}')

    ax.axvline(toy_model.char_timescales[alpha].detach(), color='g', linestyle=':', lw=2)
    # ax.quiver((toy_model.char_timescales[alpha] - toy_model.onset_times[alpha]).detach(), 0, (toy_model.char_timescales[alpha] + toy_model.onset_times[alpha]).detach(), 0, color='b')

plt.suptitle(f"Form potentials (blue) and form strengths (red) over time {details}")
plt.tight_layout()
plt.show()

# PC plots
fig, axes = plt.subplots(NUM_FEATURES, NUM_FEATURES, figsize=(5 * NUM_FEATURES, 5 * NUM_FEATURES), facecolor='white')
avg_form = torch.mean(toy_model.forms, dim=0).detach()

min_pos = positions.min(axis=0)
max_pos = positions.max(axis=0)

min_form = toy_model.forms.min(axis=0).values
max_form = toy_model.forms.max(axis=0).values

# for pc in range(NUM_FEATURES):
#     min_pos[pc] = min(min_pos[pc], min_form[pc])
#     max_pos[pc] = max(max_pos[pc], max_form[pc])

for i, j in itertools.product(range(NUM_FEATURES), range(NUM_FEATURES)):
    ax = axes[i, j]

    if i != j:
        centers = []
        for t in trange(1, NUM_STEPS-1, 2):
            # dpc1_dpc2 = (samples[t+1, [pc1, pc2]] - samples[t-1, [pc1, pc2]]) * 100
            
            # ax.quiver(samples[t, pc1], samples[t, pc2], dpc1_dpc2[0], dpc1_dpc2[1], angles='xy', scale_units='xy', scale=100, color='gray', alpha=0.1)
            center, radius = get_osculating_circle(smoothed_positions[:, [j, i]], t)
            centers.append(center)
            # circ = plt.Circle(center, radius, fill=False, color='gray', alpha=0.1)
            # ax.add_artist(circ)
            if isnan(radius):
                n_thetas = 1000
            else:
                n_thetas = min(int(radius * 100), 1000)

            x=center[0] + radius * np.cos(np.linspace(0, 2 * np.pi, n_thetas))
            y=center[1] + radius * np.sin(np.linspace(0, 2 * np.pi, n_thetas))
            ax.plot(x, y, color='gray', alpha=0.1)    

        rainbow = plt.get_cmap('rainbow')
        colors = np.linspace(0, 1, len(centers))
        colors = [rainbow(c) for c in colors]
        ax.scatter([c[0] for c in centers], [c[1] for c in centers], color=colors, s=10)

    # ax.scatter(positions[:, j], positions[:, i], cmap=rainbow, alpha=0.2)
    ax.plot(smoothed_positions[:, j], smoothed_positions[:, i], label=f'PC {j + 1} vs PC {i + 1} (smoothed)', lw=2)

    ax.set_xlabel(f'PC {j + 1}')
    ax.set_ylabel(f'PC {i + 1}')

    ax.scatter(smoothed_positions[0, j], smoothed_positions[0, i], color='g', s=50, marker='o')   
    ax.scatter(smoothed_positions[-1, j], smoothed_positions[-1, i], color='r', s=50, marker='o')

    for alpha in range(toy_model.num_forms):
        ax.scatter(toy_model.forms[alpha, j].detach(), toy_model.forms[alpha, i].detach(), color='r', marker='X', s=50)

    # ax.scatter(avg_form[j].detach(), avg_form[i].detach(), color='y')
    MULTIPLIER = 0.1

    # min_j = min(min_pos[j] * MULTIPLIER, 0, min_form[j].item())
    # max_j = max(max_pos[j] * MULTIPLIER, 0, max_form[j].item())
    # min_i = min(min_pos[i] * MULTIPLIER, 0, min_form[i].item())
    # max_i = max(max_pos[i] * MULTIPLIER, 0, max_form[i].item())
    min_j = (min_pos[j])
    max_j = (max_pos[j])
    min_i = (min_pos[i])
    max_i = (max_pos[i])
    range_j = max_pos[j] - min_pos[j]
    range_i = max_pos[i] - min_pos[i]

    ax.set_xlim(min_j - MULTIPLIER * range_j , max_j + MULTIPLIER * range_j)
    ax.set_ylim(min_i - MULTIPLIER * range_i, max_i + MULTIPLIER * range_i)
    
    # ax.set_xlim(min(min_pos[j] * MULTIPLIER, ), max(0, max_pos[j] * MULTIPLIER))
    # ax.set_ylim(min(min_pos[i] * MULTIPLIER, 0), max(0, max_pos[i] * MULTIPLIER))


fig.suptitle(f'ED plots{details}', fontsize=16)
plt.tight_layout()